#### Navigate to data/raw directory in the terminal and run: `instaloader --fast-update --no-pictures --no-videos "cattownoak"`
This downloads a separate json file for each post. Pictures and videos are not included, but url links to pictures are in the json files.

In [24]:
import os
import glob
from dotenv import find_dotenv
import lzma
import json
import re
import pandas as pd
from datetime import datetime, timedelta

In [16]:
project_dir = find_dotenv().split('.')[0]
data_dir = os.path.join(project_dir, 'data')
raw_data_dir = os.path.join(data_dir, 'raw')
processed_data_dir = os.path.join(data_dir, 'rocessed')
os.chdir(project_dir)

In [11]:
os.system('cmd /k cd Desktop\DataScience\Repos\cattown_oakland\data\raw')


32512

In [17]:
def show_dict(post_dict):
    print('type', type(post_dict))
    if isinstance(post_dict, dict):
        print(list(post_dict.keys()))
        for k in list(post_dict.keys()):
            print()
            print(k)
            show_dict(post_dict[k])
            print()
    elif isinstance(post_dict, list):
        print(post_dict)
        print(len(post_dict))
    else:
        print(post_dict)
        

In [35]:
start = datetime.now()
class post:
    
    
    def __init__(self, node):
        self.id = int(node['id'])
        self.likes = int(node['edge_media_preview_like']['count'])
        self.comments = int(node['edge_media_to_comment']['count'])
        self.timestamp = datetime.fromtimestamp(int(node['taken_at_timestamp']))
        
        def get_caption(node):
            try:
                node_caption = node['edge_media_to_caption']['edges'][0]['node']['text']
            except:
                node_caption = ''
            return node_caption
        self.caption = get_caption(node)
        
        self.hashtags = re.findall(r'(?<=#)\w+', self.caption)
        self.mentions = re.findall(r'(?<=@)\w+', self.caption)
        self.pic_dim = node['dimensions']
        self.pic_url = node['display_url']
 
        
def json_to_str(file):
    bytes_data = lzma.open(file).read()
    json_str = bytes_data.decode('utf-8')
    post_str = json.loads(json_str)
    return post_str


cattownposts_raw = pd.DataFrame()
for file in glob.glob(os.path.join(raw_data_dir, '*.json.xz')):
    post_str = json_to_str(file)
    if post_str['instaloader']['node_type'] == 'Post':
        node_info = post_str['node']
        post_dict = post(node_info).__dict__
        post_df = pd.DataFrame([post_dict])
        cattownposts_raw = pd.concat([cattownposts_raw,post_df], axis=0)
cattownposts_raw.reset_index(inplace=True, drop=True)
cattownposts_raw['date'] = cattownposts_raw['timestamp'].dt.date

end = datetime.now()
print((end-start).total_seconds())
cattownposts_raw

14.508502


,id,likes,comments,timestamp,caption,hashtags,mentions,pic_dim,pic_url,date
0,820728123025650232,153,40,2014-09-29 23:27:18,The Cat Town Cafe is happy to announce that we...,[],[],"{'height': 640, 'width': 640}",https://scontent-sjc3-1.cdninstagram.com/v/t51...,2014-09-29
1,821236589892858507,29,2,2014-09-30 16:17:32,Huge thanks to Sage & Tari from @fuming_gueril...,[],[fuming_guerilla],"{'height': 640, 'width': 640}",https://scontent-sjc3-1.cdninstagram.com/v/t51...,2014-09-30
2,822104812305642150,13,2,2014-10-01 21:02:32,Our friend and first featured artist Megan Lyn...,[],[],"{'height': 640, 'width': 640}",https://scontent-sjc3-1.cdninstagram.com/v/t51...,2014-10-01
3,823387820614611578,18,1,2014-10-03 15:31:38,#regram from Jessy O. of the Cat Zone voluntee...,[regram],[],"{'height': 640, 'width': 640}",https://scontent-sjc3-1.cdninstagram.com/v/t51...,2014-10-03
4,823918049217081998,33,3,2014-10-04 09:05:07,Happy Caturday! Are you going to #ape2014 this...,[ape2014],[],"{'height': 640, 'width': 640}",https://scontent-sjc3-1.cdninstagram.com/v/t51...,2014-10-04
...,...,...,...,...,...,...,...,...,...,...
4070,2283095831102249903,672,10,2020-04-08 15:50:53,Feeling tired this afternoon? Follow the advic...,"[cattownalum, WhereAreTheyNowWednesday, adopte...",[],"{'height': 640, 'width': 640}",https://scontent-sjc3-1.cdninstagram.com/v/t51...,2020-04-08
4071,2283690822166095775,327,1,2020-04-09 11:33:02,During Easter and Passover we like to adorn ou...,"[catsafety, catsafeplants, catsafe, flowersand...",[],"{'height': 480, 'width': 480}",https://scontent-sjc3-1.cdninstagram.com/v/t51...,2020-04-09
4072,2284278438260672031,702,5,2020-04-10 07:00:31,Today we bring you an adorable photo of a very...,"[catnap, cattown, felinefriday, catsagram, cat...",[blangstedl],"{'height': 1080, 'width': 1080}",https://scontent-sjc3-1.cdninstagram.com/v/t51...,2020-04-10
4073,2285025847664854551,413,3,2020-04-11 07:45:29,Want to indulge in some shopping therapy while...,"[cattownoak, cattowncat, cattownoakland, nonpr...",[],"{'height': 1080, 'width': 1080}",https://scontent-sjc3-1.cdninstagram.com/v/t51...,2020-04-11


In [18]:
n = 4054

counter = 0
for file in glob.glob(os.path.join(raw_data_dir, '*.json.xz')):
    counter += 1
    if counter == n+1:
        print(counter)
        post_str = json_to_str(file)
        show_dict(post_str)
    if counter == n+2:
        break

4055


NameError: name 'json_to_str' is not defined

In [38]:
cattownposts_raw[cattownposts_raw['caption'].str.contains('Silverado')]

,id,likes,comments,timestamp,caption,hashtags,pic_dim,pic_url,date
4054,2274293310015432405,485,3,2020-03-27 12:21:51,"Isn't is nice to get real, handwritten mail in...","[cattownalum, cattowncat, tiggerthecat, catsof...","{'height': 1080, 'width': 1080}",https://scontent-sjc3-1.cdninstagram.com/v/t51...,2020-03-27


In [3]:
list(set([item for sublist in cattownposts_raw['hashtags'].tolist() for item in sublist]))

['adoptablecats',
 'gamenight',
 'catyoga',
 'youdidit',
 'catartist',
 'fbf',
 'tshirtdesign',
 'tortoiseshellcat',
 'catlitter',
 'cookiecat',
 'newatcattown',
 'artsubmission',
 'catcentral',
 'HappyBirthdayMaddie',
 'sleepingcat',
 'comeforthecoffeestayforthecats',
 'tortoisecat',
 'lukathecat',
 'catcontest',
 'oaklandrescue',
 'catillustration',
 'snugglecat',
 'happynationaldogday',
 'blackfriday',
 'Caturday',
 'adoptables',
 'seniorpetmonth',
 'NationalAnimalShelterAppreciationWeek',
 'ringworm',
 'yogaoakland',
 'buylocal',
 'declaw',
 'cattrivia',
 'irestmeditation',
 'donateoakland',
 'graffiticats',
 'relaxwithacat',
 'Regram',
 'tactica',
 'jaxthecat',
 'straycat',
 'seniorcatsunday',
 'happyfamily',
 'cattowncats',
 'uptownoakland',
 'loveoakland',
 'gardencat',
 'thankstoyou',
 'bayareaevents',
 'cupcakecatyrday',
 'AdoptAGangstaPet',
 'bekind',
 'peptalk',
 'aishathecat',
 'tabbytown',
 'catsandtreats',
 'annathecat',
 'blackandwhitecat',
 'octoberadoptions',
 'meetcoo

In [5]:
cattownposts_raw.columns

Index(['id', 'likes', 'comments', 'timestamp', 'caption', 'hashtags',
       'pic_dim', 'pic_url', 'date'],
      dtype='object')